In [1]:
# Major Libraries
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [2]:
dataset = pd.read_csv("diabetic_data.csv", na_values="?")

C:\Users\CHANDIMA\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(type(dataset))

<class 'pandas.core.frame.DataFrame'>


In [4]:
print(dataset.shape)

(101766, 50)


Categorical data should be encoded.


In [5]:
#Remove useless features
dataset.drop(['encounter_id', 'patient_nbr',  'weight', 'payer_code',  'medical_specialty', 
        'number_inpatient', 'number_outpatient', 'number_emergency',  'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'glyburide_metformin', 'glipizide_metformin', 'glimepiride_pioglitazone', 'metformin_rosiglitazone', 'metformin_pioglitazone' ], axis=1, inplace=True)

In [6]:
print(dataset.columns)

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'insulin', 'change',
       'diabetesMed', 'readmitted'],
      dtype='object')


These 22 features are considered in the final analysis.

In [7]:
dataset.shape

(101766, 18)

In [8]:
print(dataset.isnull().sum())

race                        2273
gender                         0
age                            0
admission_type_id              0
discharge_disposition_id       0
admission_source_id            0
time_in_hospital               0
num_lab_procedures             0
num_procedures                 0
num_medications                0
diag_1                        21
diag_2                       358
diag_3                      1423
number_diagnoses               0
insulin                        0
change                         0
diabetesMed                    0
readmitted                     0
dtype: int64


In [9]:
# In the feature "gender", there are 03 invalid data which should be removed.
dataset = dataset[dataset.gender != 'Unknown/Invalid']

In [10]:
finalDataset = dataset.dropna()

In [11]:
finalDataset.shape

(98052, 18)

In [12]:
# ICD9 Codes for diag_1, diag_2 and diag_3
def ICD9_classifier(diagnoses):
    # Returns a series of strings corresponding to type of ICD9 diagnosis
    # diagnoses is a list
    gr_diagnoses = diagnoses.copy()
    icd9_dct = {
                'Infectious':(1, 139),
                'Neoplasmic':(140,239),
                'Hormonal':(240, 279),
                'Blood':(280,289),
                'Mental':(290,319),
                'Nervous':(320,359),
                'Sensory':(360,389),
                'Circulatory':(390,459),
                'Respiratory':(460,519),
                'Digestive':(520,579),
                'Genitourinary':(580,629),
                'Childbirth':(630,679),
                'Dermatological':(680,709),
                'Musculoskeletal':(710,739),
                'Congenital':(740,759),
                'Perinatal':(760,779),
                'Miscellaneous':(780,799),
                'Injury':(800,999)
               }
    for i, diagnosis in enumerate(diagnoses):
        if (str(diagnoses[i])[0] == 'E') or (str(diagnoses[i])[0] == 'V'):
            gr_diagnoses[i] = 'Accidental'
        elif (str(diagnoses[i]).lower() == 'nan'):
            gr_diagnoses[i] = 'NaN'
        else:
            for key, icd_range in icd9_dct.items():
                if (int(float(diagnoses[i])) >= icd_range[0]) and (int(float(diagnoses[i])) <= icd_range[1]):
                    gr_diagnoses[i] = key
    return gr_diagnoses

In [13]:
d1 = ICD9_classifier(finalDataset.diag_1.values)
d2 = ICD9_classifier(finalDataset.diag_2.values)
d3 = ICD9_classifier(finalDataset.diag_3.values)


finalDataset.diag_1 = d1
finalDataset.diag_2 = d2
finalDataset.diag_3 = d3

C:\Users\CHANDIMA\Anaconda3\lib\site-packages\pandas\core\generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [14]:
# Label-encode age feature to an integer in the center of the raw bin
finalDataset.age = (LabelEncoder().fit_transform(finalDataset.age)*10) + 5

#  Encoding - Categorical Data
lb = LabelEncoder()
finalDataset['race'] = lb.fit_transform(finalDataset['race']) 
finalDataset['gender'] = lb.fit_transform(finalDataset['gender'])
finalDataset['diag_1'] = lb.fit_transform(finalDataset['diag_1']) 
finalDataset['diag_2'] = lb.fit_transform(finalDataset['diag_2'])
finalDataset['diag_3'] = lb.fit_transform(finalDataset['diag_3'])
finalDataset['insulin'] = lb.fit_transform(finalDataset['insulin']) 
finalDataset['change'] = lb.fit_transform(finalDataset['change'])
finalDataset['diabetesMed'] = lb.fit_transform(finalDataset['diabetesMed'])

le_readm = LabelEncoder()
finalDataset.readmitted = pd.Series(le_readm.fit_transform(finalDataset.readmitted),
                                                      index=finalDataset.index)



C:\Users\CHANDIMA\Anaconda3\lib\site-packages\pandas\core\generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\CHANDIMA\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\CHANDIMA\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/panda

In [15]:
num_features = finalDataset.columns

# Remove outliers by class
for response in finalDataset.readmitted.unique(): # For each readmission response
    response_df = finalDataset[finalDataset.readmitted==response] # For all samples that meet the readmission criteria
    for column in response_df[num_features].columns: # Iterate through each feature of the sample
        Q1 = response_df[column].quantile(0.25)
        Q3 = response_df[column].quantile(0.75)
        IQR = Q3-Q1
        if IQR > 0: # And remove features that are outside of Q1 - 1.5IQR or Q3 + 1.5IQR
            response_df = response_df[(response_df[column] > Q1-(1.5*IQR)) & (response_df[column] < Q3+(1.5*IQR))]
            finalDataset = finalDataset.drop(finalDataset[finalDataset.readmitted==response].index, axis=0)
            finalDataset = finalDataset.append(response_df)

In [16]:
onehotencoder = OneHotEncoder(categorical_features = [17])
finalDataset = onehotencoder.fit_transform(finalDataset).toarray()

In [17]:
X = finalDataset[:, 0:17]
y = finalDataset[:, 17:]

In [18]:
X

array([[  0.,   1.,   0., ...,  16.,   8.,   8.],
       [  0.,   1.,   0., ...,   3.,  10.,   9.],
       [  0.,   1.,   0., ...,   7.,   8.,   3.],
       ..., 
       [  1.,   0.,   0., ...,   3.,   8.,   9.],
       [  1.,   0.,   0., ...,   8.,   3.,   9.],
       [  1.,   0.,   0., ...,   6.,   8.,   9.]])

In [19]:
y

array([[ 1.,  1.,  1.],
       [ 2.,  1.,  1.],
       [ 2.,  1.,  1.],
       ..., 
       [ 2.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.]])

In [20]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [21]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [22]:
# Making Models
# Making The ANN

#  Importing the keras libraries & packages
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [23]:
#  Initializing the ANN
classifier = Sequential()

In [24]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(activation = "relu", input_dim = 17, units = 10, kernel_initializer = "uniform"))

In [25]:
#  Adding the second hidden layer
classifier.add(Dense(activation = "relu", units = 10, kernel_initializer = "uniform"))

In [26]:
#  Adding the output layer
classifier.add(Dense(activation = "softmax", units = 3, kernel_initializer = "uniform")) 

In [27]:
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [28]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
44796/44796 [==============================] - 8s 190us/step - loss: 2.7551 - acc: 0.8855
Epoch 2/100
44796/44796 [==============================] - 6s 134us/step - loss: 2.7432 - acc: 0.8855
Epoch 3/100
44796/44796 [==============================] - 6s 134us/step - loss: 2.7418 - acc: 0.8855
Epoch 4/100
44796/44796 [==============================] - 6s 136us/step - loss: 2.7412 - acc: 0.8855
Epoch 5/100
44796/44796 [==============================] - 6s 137us/step - loss: 2.7409 - acc: 0.8855
Epoch 6/100
44796/44796 [==============================] - 6s 137us/step - loss: 2.7404 - acc: 0.8855
Epoch 7/100
44796/44796 [==============================] - 6s 136us/step - loss: 2.7401 - acc: 0.8855
Epoch 8/100
44796/44796 [==============================] - 6s 138us/step - loss: 2.7400 - acc: 0.8855
Epoch 9/100
44796/44796 [==============================] - 7s 152us/step - loss: 2.7397 - acc: 0.8855
Epoch 10/100
44796/44796 [==============================] - 193s 4ms/step - loss: 

44796/44796 [==============================] - 7s 163us/step - loss: 2.7365 - acc: 0.8854
Epoch 81/100
44796/44796 [==============================] - 9s 190us/step - loss: 2.7363 - acc: 0.8853
Epoch 82/100
44796/44796 [==============================] - 6s 142us/step - loss: 2.7364 - acc: 0.8851
Epoch 83/100
44796/44796 [==============================] - 6s 136us/step - loss: 2.7365 - acc: 0.8852
Epoch 84/100
44796/44796 [==============================] - 6s 131us/step - loss: 2.7365 - acc: 0.8853
Epoch 85/100
44796/44796 [==============================] - 6s 131us/step - loss: 2.7365 - acc: 0.8854
Epoch 86/100
44796/44796 [==============================] - 6s 129us/step - loss: 2.7364 - acc: 0.8853
Epoch 87/100
44796/44796 [==============================] - 6s 137us/step - loss: 2.7363 - acc: 0.8854
Epoch 88/100
44796/44796 [==============================] - 6s 132us/step - loss: 2.7364 - acc: 0.8854
Epoch 89/100
44796/44796 [==============================] - 6s 129us/step - loss: 2.73

In [30]:
scores = classifier.evaluate(X_test, y_test)
print("%s: %.2f%%" % (classifier.metrics_names[1], scores[1]*100))

14932/14932 [==============================] - 0s 25us/step
acc: 88.08%


In [31]:
scores = classifier.evaluate(X_train, y_train)
print("%s: %.2f%%" % (classifier.metrics_names[1], scores[1]*100))

44796/44796 [==============================] - 1s 31us/step
acc: 88.55%
